# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.4.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
# We want to apply transformations on the images of the training set. The reason is that we want to avoid overfitting
# i.e. getting very high accuracy on the training set vs. testing set (This is a common practice in Computer Vision)
# What we are going to do: "Image Augmentation": Zoom Range, Shear Range, Horizontal Flip --> We are using those transformations
# in particular because they are used in the code snippit in Keras documentation
# rescale: applies feature scaling on the image features (normalization)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Applying the ImageDataGenerator object on the training set (the transformations)
# target_size: reducing the size of the images to reduce training time. (64, 64) is just an emperical
# number that gives good results without sacrificing quality
# batch_size: the transformation is going to be applied on batches of input images. 
# 32 is a known good number for that (a classic default value)
# class_mode: of the output. can be 'binary' or 'categorical'. We choose binary, because we are going to choose
# between two output classes: cat or dog
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')


### Preprocessing the Test set

In [ ]:
# For the testset, we don't want to apply the same transformations we applied on the training set because they are supposed 
# to be totally new images to the model. But we just want to make sure we do the feature scaling. Therefore, the ImageDataGenerator
# will only the rescale parameter set to 1./255 (normalization)
testset_datagen = ImageDataGenerator(rescale=1./255)

# Applying the ImageDataGenerator object on the testset
test_set = testset_datagen.flow_from_directory('dataset/testset',
                                               target_size=(64, 64),
                                               batch_size=32,
                                               class_mode='binary')

## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential() # Initializes the cnn as a sequence of layers (as opposed to Computational Graph)

### Step 1 - Convolution

In [ ]:
# Adding a Convolution layer to the cnn
# filters: the number of filters (feature detectors) that we want to apply during convolution. 32 is a known value that works well
# 32 in the first layer
# kernel_size: the dimensions of the filter (it's a square matrix)
# activation: the activation function
# input_shape: the shape of the input which is 64x64x3 (3 because they are colored images: RGB)
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [ ]:
# Adding Max Pooling layer to generate the Pooled Feature Map
# pool_size: the size of the frame that is applied on the feature map (it's a square)
# strides: the step by which the pool is moving along the feature map
# padding: can be either 'valid' default or 'same'. We are going to use the default. This is how to treat the pixels when the
# pool has column outside the feature map (when it's moved to the right). 'valid' ignores the column that exists outside the feature map,
# 'same' sets the values in this column to zero. Both options don't create much difference
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [ ]:
print(prediction)